In [46]:
from hidromet import config
from hidromet import utils

import geopandas as gpd
from pathlib import Path
import pandas as pd
import json


In [47]:
gdf = gpd.read_file(config.dir_contorno)
crs = gdf.crs
bacia = gdf.iloc[0]
nome_bacia = bacia.bacia

### Remover repetidos

In [48]:
def verificar_postos_duplicados(df:pd.DataFrame) -> pd.DataFrame:
    """
    Verifica se há postos duplicados.
    """
    
    return df[df.duplicated(['latitude', 'longitude'], keep=False)]


In [49]:
arquivo_coords_inmet = config.dir_prec_inmet.joinpath(f"coords_{nome_bacia}.json")
arquivo_serie_inmet = config.dir_prec_inmet.joinpath(f"{nome_bacia}.csv")

json_inmet = utils.carregar_json(arquivo_coords_inmet)
postos_ok = json_inmet['postos_ok']
df_inmet = pd.DataFrame.from_dict(postos_ok).round(2)

series_bacia_inmet = pd.read_csv(arquivo_serie_inmet, index_col=0)
postos_inmet = df_inmet.codigo
serie_inmet = series_bacia_inmet[postos_inmet]

In [50]:
arquivo_serie_ana = config.dir_prec_ana.joinpath(f"{nome_bacia}.csv")
arquivo_coords_ana = config.dir_prec_ana.joinpath(f"coords_{nome_bacia}.json")

json_ana = utils.carregar_json(arquivo_coords_ana)
postos_ok = json_ana['postos_ok']
df_ana = pd.DataFrame.from_dict(postos_ok).round(2)

series_bacia_ana = pd.read_csv(arquivo_serie_ana, index_col=0)
postos_ana = df_ana.codigo
serie_ana = series_bacia_ana[postos_ana]

In [51]:
df_unificado = pd.concat([df_ana, df_inmet])
postos_duplicados = verificar_postos_duplicados(df_unificado)

df_unificado.set_index('codigo', inplace=True)

In [52]:
duplicados_agrupados = postos_duplicados.groupby(["latitude", "longitude"])
grupos = duplicados_agrupados.groups.keys()

for duplicado in grupos:
  duplicados = duplicados_agrupados.get_group(duplicado)

In [53]:
serie_duplicada = pd.DataFrame()

for ix, grupo in enumerate(grupos):
    duplicados = duplicados_agrupados.get_group(grupo)

    if duplicados.iloc[ix].codigo in postos_ana.values:
        serie_duplicada[duplicados.iloc[ix].codigo] = serie_ana[duplicados.iloc[ix].codigo]
        
    else:
        serie_duplicada[duplicados.iloc[ix].codigo] = serie_inmet[duplicados.iloc[ix].codigo]

    
    falha = serie_duplicada.isna().sum()/len(serie_duplicada)
    codigo_drop = falha.loc[falha.values == falha.max()].index.item()

    if codigo_drop in postos_ana.values:
        serie_ana.drop(codigo_drop, axis=1, inplace=True)
    else:
        serie_inmet.drop(codigo_drop, axis=1, inplace=True)

    
    df_unificado.drop(codigo_drop, axis=0, inplace=True)

df = pd.concat([serie_ana, serie_inmet], axis=1)
df.to_csv(f"{config.dir_prec_concat}/{nome_bacia}.csv")

In [54]:
df_unificado_json = df_unificado.reset_index().to_dict(orient='records')

arquivo_json = Path(config.dir_prec_concat, f'coords_{nome_bacia}.json')
with open(arquivo_json, 'w') as f:
    json.dump(df_unificado_json, f)